In [1]:
from allennlp_models import pretrained
import re
allenslr=pretrained.load_predictor('structured-prediction-srl-bert')

2022-09-14 14:53:29.613919: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-14 14:53:29.613933: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
lerc is not a registered model.
visual-entailment is not a registered model.
vqa_vilbert is not a registered model.


# Improvements in labeler algorithm from the study of the PropBank anotation rules 

#### ¿Qué es PropBank?
- Es un cuerpo anotado con proposiciones verbales y sus argumentos
- Es un recurso orientado a verbos
- PropBank no anota enventos o estados de las cosas descritas usando sustantivos
- PropBank anota todos los verbos encontrados en un corpus

**Necesita que todos los argumentos de un verbo sean constituyentes sintácticos y los diferentes sentidos de una palabra solo se distinguen si las diferencias se relacionan con los argumentos**

#### ¿Qué son los argumentos de un verbo?
Entendermos los argumentos como las _palabras necesarias_ para un correcto entendimiento (gramatical) de las frases. 

#### ¿Qué son los constituyentes sintácticos?
Son una palabra o conjunto de palabras que funcionan como una sola unidad dentro de una estructura jerárquica.

## Working with AllenNLP
AllenNLP identificará cada uno de los verbos (y modales) presentes en la oración y para cada uno de ellos intentará clasificar las demás palabras en la oración si las reconoce como argumentos válidos. Esto supone un primer problema a la hora de utilizar la herramienta, pues la salida del predictor dada una oración tendrá identificaciones correctas e incorrectas de verbos y argumentos. Por ejemplo, de la oración

_If you liked the music we were playing last night, you will absolutely love what we're playing tomorrow!_

El modelo distinguirá un total de cinco verbos
- liked
- were
- playing
- will
- love
- are
- playing

Para cada uno de los verbos, AllenNLP buscará los argumentos verbales y procederá a anotarlos.

Dada la oración y la lista anterior de verbos, podemos augurar que las peores anotaciones se darán cuando AllenNLP intente anotar los argumentos para _were_, _will_ y _are_

In [ ]:
sentence="If you liked the music we were playing last night, you will absolutely love what we're playing tomorrow!"
output=allenslr.predict(sentence)
print("")
print("verb: were")
print(output['verbs'][1]['description'])
print("verb: will")
print(output['verbs'][3]['description'])
print("verb: are")
print(output['verbs'][5]['description'])
print("")

En estos casos, el etiquetador fue incapaz de ir más allá de un reconocimiento incorrecto de los verbos.
Veamos que las anotaciones mejoran cuando reconocemos correctamente los verbos y no los modales o auxiliares

In [ ]:
print("")
print("liked")
print(output['verbs'][0]['description'])
print("playing")
print(output['verbs'][2]['description'])
print("love")
print(output['verbs'][4]['description'])
print("playing")
print(output['verbs'][6]['description'])
print("")

En este caso el etiquetador realiza  anotaciones, distinguiendo los distintos tipos de argumentos
- ARG0
- ARG1
- ARG2...
- ARGM-

## Etiquetado

De acuerdo con las reglas de PropBank, las etiquetas de los argumentos dependen de los roles semánticos posibles, estas son etiquetadas como ARG0, ARG1, ARG2,... El prefijo ARGM- se refiere a modificadores del verbo temporales, de locación, de modo y demás.

Por ejemplo, para el verbo play se etiqueta como `ARG0` al sujeto que toca "we", y como `ARG1` al objeto tocado "the music". Finalmente como modificador temporal el conjunto "last night". Notemos que bajo esta elección varias partes de la oración quedan sin ser anotadas. De acuerdo a las reglas de PropBank, éstas son las que no se consideran argumentos del verbo anotado.

Otro ejemplo de esta misma situación se da cuando se vuelve a etiquetar como verbo "playing" pero ahora se reconoce como `ARG1` a la palabra "what" (lo que tocaremos mañana)

De esta forma además del problema de la distinción correcta entre verbos, modales y auxiliares, está el problema de mutilación o pérdida de información por el etiquetado, que depende de los roles semánticos y argumentos válidos para el verbo seleccionado.

En AllenNLP, aquellas palabras que no hayan podido ser anotadas se quedan con el tag 'O'

In [ ]:
output['verbs'][1]

## FrameSets

Para algunos verbos, existen distintas interpretaciones (FrameSets) que se pueden dar en oraciones, dando lugar a distintos roles semánticos posibles. Por ejemplo con el verbo "leave"

In [ ]:
sentence="Mary left the room"
output=allenslr.predict(sentence)
print("")
print(output['verbs'][0]['description'])
print("ARG0 sujeto que se va")
print("ARG1 objeto dejado")
print("")
sentence="Mary left her daughter-in-law her pearls in her will"
output=allenslr.predict(sentence)
print(output['verbs'][0]['description'])
print("ARG0 sujeto que da")
print("ARG1 objeto dado")
print("ARG2 beneficiario")
print("ARGM modo")
print("")

En ambos casos después de identificar el verbo, AllenNLP es capaz de distinguir los distintos FrameSets y realizar las anotaciones correctas. 

Otras formas en como se pueden tener distintos FrameSets de distintos verbos es agregando una partícula, modificándose los roles semánticos. Por ejemplo tomando el verbo "keep" a "keep up":

In [ ]:
sentence="John can't keep up with Mary's rapid mood swings."
output=allenslr.predict(sentence)
output['verbs'][1]['description']

AllenNLP puede reconocer correctamente los argumentos excepto a la partícula.

## Elecciones de argumentos

- ARG0 es asignado a argumentos que se entienden como agentes causantes o experimentadores
- ARG1 es asaignado al paciente, el objeto que cambia de estado o está siendo afectado por la acción

Dependiendo de los verbos y los roles, distintos objetos pueden clasificarse como distintos argumentos verbales. (objetos que pueden ser tomados en la misma oración tanto como objetos como pacientes). La regla de etiquetado señala que siempre debe ser etiquetado el de más alto rango. ARG0 > ARG1 > ARG2 ...

En resumen, ARG0 son los argumentos que provocan la acción denotada por el verbo, ya sea de forma agencial o no, así como los que tradicionalmente se clasifican como experimentadores, es decir, los argumentos de verbos estativos como amor, odio, miedo. ARG1, por otro lado, son aquellos que cambian debido a una causalidad externa, así como otros tipos de argumentos "afectados"

Los argumentos modificadores usados en PropBank son

- DIR: Directionals
- LOC: Locatives
- MNR: Manner
- EXT: Extent
- REC: Reciprocals
- PRD: Secondary Predication
- PNC: Purpose
- CAU: cause
- DIS: discourse
- ADV: adverbials
- MOD: modals
- NEG: negation
- DSP: direct speech
- RLC: relative clauses

AllenNLP es capaz de reconocerlos y son etiquetados comenzando con el prefijo ARGM- facilitando su identificación

## Comentarios Finales sobre AllenNLP

AllenNLP, es capaz de reconocer correctamente el FrameSet "involucrado" en la oración, y con la elección correcta de verbo, a los sujetos y objetos en la oración, siguiendo todas las reglas de PropBank. Además es capaz de etiquetar correctamente oraciones pasivas. Es importante recalcar que oraciones carentes de ARG0 o ARG1 no son necesariamente etiquetados incorrectos, pues pueden estár implícitos en la oración. Pueden incluso estar presentes en una frase etiquetas ARG1 y ARG2 sin la necesidad de un ARG0. Como ejemplos:

In [ ]:
sentence="John was hit by Mary"
output=allenslr.predict(sentence)
print("")
print("Oración pasiva")
print(output['verbs'][1]['description'])
sentence="John was hit"
output=allenslr.predict(sentence)
print("")
print("Oración sin ARG0")
print(output['verbs'][1]['description'])
sentence="Was acussed of conducting illegal business and possessinf illegal materials"
output=allenslr.predict(sentence)
print("")
print("Oración con ARG2 únicamente")
print(output['verbs'][1]['description'])
print("El acusado (ARG1) está implícito en 'was' y el acusador (ARG0) no está presente en la oración.")

## Algoritmo

De todos los comentarios anteriores la tarea a lograr es identificar correctamente al verbo del listado que realiza AllenNLP. A partir de ahí, la creación de las tuplas (Sujeto, Objeto, Predicado) se realiza asumiendo que el etiquetado (la identificación de roles semánticos) de AllenNLP es correcto.

**Proposición 1**: La creación de tuplas es posible si existen al menos dos etiquetas en la oración.

La necesidad de este requerimiento es explicada si observamos que anotar únicamente al verbo nos deja sin posibilidades de distinguir sujetos, objetos y modificadores de entre todas las demás palabras al carecer de identificadores. Si existen al menos dos etiquetas, entonces es posible crear una tupla con al menos esas dos partes de la oración. Esto nos motiva a la siguiente definición

**Definición 1**: Diremos que el etiquetado de una oración es valido si este contiene al menos dos etiquetas.

Regresando a la salida de AllenNLP, el listado de posibles etiquetados depende de la _"cantidad de verbos"_  presentes en la oración. Ese listado puede ser de tamaño cero, es decir no ha reconocido ningún verbo, de tamaño uno, reconociendo un verbo únicamente o de tamañano mayor a uno.

En el caso cero, no es posible crear ninguna tupla y el algoritmo debe lanzar una excepción.

En el caso uno, el algoritmo debe verificar el etiquetado de la oración y si es válido, crear la tupla, de lo contrario lanzar una excepción.

En el caso mayor a uno, el algoritmo debe verificar la validez del etiquetado. Funcionando como un primer filtro contra las anotaciones incorrectas de verbos.

Una primera implementación en pseudocódigo:
```
function algorithm(sentence):
    
    output=AllenNLP(sentence)
    n_verbs= call calculate_number_of_verbs(output)
    
    if(n_verbs == 0){
        throw exception
    }
    
    if(n_verbs == 1){
        if(validate(output)){
            create_tuple(output)
        }
        else{
            throw exception
        }   
    }
    
    if (n_verbs >= 1){
        for verbs in n_verbs{
            validate(output(verb))
        }
    }
```
De tener más de un verbo anotado, seleccionaremos como la mejor anotación aquella que posea la menor cantidad de palabras sin etiquetar. Es decir el mejor verbo será aquel para el que exista una mayor cantidad de palabras etiquetadas como argumentos.

Como nota, no es recomendable proponer un filtro que busque al mejor verbo anotado buscando la propuesta que tenga a absolutamente todas las palabras como argumentos, porque partimos del supuesto AllenNLP posee limitaciones en el etiquetado pudiendo omitir al menos alguna vez partículas, contracciones, modales y/o auxiliares.

Con este método, podemos seleccionar la mejor anotación y crear la tupla necesaria, cubriendo todos los casos.

### La salida de AllenNLP:

```python
output['verb'] #es una lista de tamaño n con n verbos identificados. Cada elemento de la lista es un diccionario
output['verb'][i] #es un diccionario. Tiene las claves
{'verb': str,
 'description': str
 'tags': list
}
```
Podemos usar `'tags'` para medir la cantidad de palabras que no pudieron ser etiquetadas y realizar las validaciones, pero es mejor realizar las extracciones a partir de la clave `'description'`

In [49]:
import string
def extract(sentence):
    """
    from a sentence use AllenNLP SRLtool to extrac (subject-object, predicate) tuples
    
    Arguments
        sentence :: str
    
    Returns:
        If possible
        (str, str) list 
        Else
        None
    """
    remove='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~'
    input=sentence.translate(str.maketrans('', '', remove))
    count=[]                                        # num de palabras sin anotar por verbo
    counter=0
    output=allenslr.predict(input)

    if len(output['verbs'])==0:                     # Ningun verbo anotado
        print("ERROR: Ningún verbo anotado")
        return None
    elif len(output['verbs'])==1:                   # Un verbo anotado
        tags=output['verbs'][0]['tags']             # Valida el etiquetado
        if bool(re.search('ARG[0-5]', string) for string in tags):
            tuple=create_tuples(output)             # Extracción de información
        else:
            print("ERROR: Insuficientes etiquetas para crear la tupla")
            return None
    else:                                           # Más de un verbo anotado
        for verb in output['verbs']:                # Valida eitquetas para cada verbo
            for tag in verb['tags']:
                if tag == 'O':
                    counter+=1                      # Cuenta las palabras sin anotar
            count.append(counter)
            counter=0
        tags=output['verbs'][search(count)]['tags']
        if bool(re.search('ARG[0-5]', string) for string in tags):
            tuple=create_tuples(output, search(count))  # Extracción de información
        else:
            print("ERROR: Insuficientes etiquetas para crear la tupla")
            return None
    return tuple
def search(a):
    """
    search algorithm to find the smallest amount of unlabel words

    Args:
    a :: list

    Return
    index of  the list :: int
    """
    j=0
    for i in range(1,len(a)):
        if a[i]<a[j]:
            j=i
    return j
def create_tuples(output, index=0):
    """
    Create a tuple from PropBank Anotations

    Arg:
    output :: Dict create with AllenNLP SLR
    index :: int to acces to the correct labeling 
    
    Returns:
    (sj_oj, prd) list
    """
    SP=[]
    O=[]
    tags=output['verbs'][index]['tags']
    words=output['words']
    for a in range(0,5):
        if any('ARG'+str(a) in string for string in tags):
            for i in range(len(tags)):
                if bool(re.search('ARG['+str(a+1)+'-5]', tags[i])):
                    O.append(words[i])
                elif not 'O' in tags[i]:
                    SP.append(words[i])
            break
    SP=" ".join(SP)
    O=" ".join(O)
    return [SP,O]

In [8]:
sentence="If you liked the music we were playing last night, you will absolutely love what we're playing tomorrow!"
output=allenslr.predict(sentence)
txt=output['verbs'][4]['description']
print(txt)
print(output['words'])

[ARGM-ADV: If you liked the music we were playing last night] , [ARG0: you] [ARGM-MOD: will] [ARGM-ADV: absolutely] [V: love] [ARG1: what we 're playing tomorrow] !
['If', 'you', 'liked', 'the', 'music', 'we', 'were', 'playing', 'last', 'night', ',', 'you', 'will', 'absolutely', 'love', 'what', 'we', "'re", 'playing', 'tomorrow', '!']


In [50]:
extract(sentence)

['If you liked the music we were playing last night you absolutely love',
 "what we 're playing tomorrow"]

In [3]:
sentence1="Diabetes is a noncommunicable disease"
sentence2="The cause of lung cancer can be DNA methylation"
sentence3="Cystic fibrosis is an example of an inherited disease that is caused by a mutation on a gene"

In [54]:
sentence4="More than a few CEOs say the red-carpet treatment tempts them to return to a heartland city for future meetings."
extract(sentence4)

['More than a few CEOs say',
 'the redcarpet treatment tempts them to return to a heartland city for future meetings']